Followed tutorial from https://colab.research.google.com/github/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb#scrollTo=XwOdzQ_PAJi0
 
Also, reviewed SparkNLP documentation to understand each step more deeply


In [1]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:4.3.1",
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type": "native",
        "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"

    }
}

In [2]:
sc.install_pypi_package('spark-nlp',"https://pypi.org/simple")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1716401927822_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/13/96/a580e098e00905ef715253fc85589db00ca5bfa324deb5aa7cb4fc069004/spark_nlp-5.3.3-py2.py3-none-any.whl

In [3]:
#Ensure executers are using enough memory
sc._conf.get('spark.executor.memory')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'18971M'

In [4]:
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df = spark.read.parquet("s3://finalproject-nat-s3/prepared-data/*.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#Recall structure of the dataframe 
print('Total Columns: %d' % len(df.dtypes))
print('Total Rows: %d' % df.count())
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 11
Total Rows: 1646407
root
 |-- id: string (nullable = true)
 |-- created: string (nullable = true)
 |-- author: string (nullable = true)
 |-- score: string (nullable = true)
 |-- title: string (nullable = true)
 |-- selftext: string (nullable = true)
 |-- num_comments: string (nullable = true)
 |-- entire_text: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- person_1: string (nullable = true)
 |-- person_2: string (nullable = true)

In [7]:
#Recall structure of the dataframe 
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+--------------------+-----+--------------------+--------------------+------------+--------------------+----+--------+--------+
|     id|         created|              author|score|               title|            selftext|num_comments|         entire_text|year|person_1|person_2|
+-------+----------------+--------------------+-----+--------------------+--------------------+------------+--------------------+----+--------+--------+
| kj1flj|2020-12-23 15:12|    u/throwaway_135i|    2|I 20M am moving 1...|Hey rrelationship...|          13|i 20m am moving 1...|2020|        |        |
| qoi622|2021-11-07 00:40|           u/fatiabs|    7|My parents 45F 56...|I 22F have had a ...|           3|my parents 45f 56...|2021|      45|      56|
| 1kbqs9|2013-08-13 22:28|         u/[deleted]|    2|My 19F mom caught...|Hello   So this m...|           6|my 19f mom caught...|2013|        |        |
| gwqxas|2020-06-04 16:41|u/ThrowRAmattmurdock|    1|I22M am having do...|I want t

In [8]:
#Start with DocumentAssembler --> This prepares the data into a format that 
#SparkNLP can understand

#https://sparknlp.org/api/com/johnsnowlabs/nlp/DocumentAssembler

documentAssembler_text = DocumentAssembler()\
     .setInputCol("entire_text")\
     .setOutputCol('document_text')

#documentAssembler_title = DocumentAssembler()\
     #.setInputCol("title")\
     #.setOutputCol('document_title')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Tokenizer --> Separates sentence into words

#https://sparknlp.org/api/com/johnsnowlabs/nlp/annotators/Tokenizer.html

tokenizer = Tokenizer() \
            .setInputCols(['document_text'])\
            .setOutputCol('tokenized')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#Normalizer --> cleans the tokens 

#https://sparknlp.org/docs/en/annotators#normalizer

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Lemmatizer --> converts words of the same family into a common root word. SparkNLP
#provides an already pretrained lemmatization model

#https://sparknlp.org/api/com/johnsnowlabs/nlp/annotators/LemmatizerModel.html

lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

In [12]:
#Stop words
#Asked ChatGPT for a list mentioning that there are a lot of submissions that 
#have abbreviated words like didn't, etc. but without the apostrophe

stop_words = [
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", 
    "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", 
    "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", 
    "did", "doing", "will", "would", "should", "can", "could", "may", "might", "must", "shall", "ought", "about", "above", "across", "after", 
    "against", "along", "amid", "among", "around", "as", "at", "before", "behind", "below", "beneath", "beside", "between", "beyond", "but", 
    "by", "concerning", "considering", "despite", "down", "during", "except", "for", "from", "in", "inside", "into", "like", "near", "next", 
    "notwithstanding", "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past", "regarding", "round", "since", "than", "through", 
    "throughout", "till", "to", "toward", "towards", "under", "underneath", "unlike", "until", "up", "upon", "versus", "via", "with", "within", 
    "without", "cant", "cannot", "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", "havent", "hed", "hell", "hes", "howd", 
    "howll", "hows", "id", "ill", "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", "mustnt", "shant", "shed", "shell", 
    "shes", "shouldve", "shouldnt", "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", "theyll", "theyre", "theyve", "wed", 
    "well", "were", "weve", "werent", "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", "whens", "whered", "wherell", "wheres", 
    "whichd", "whichll", "whichre", "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", "whys", "wont", "wouldve", "wouldnt", 
    "youd", "youll", "youre", "youve", "f", "m", "relationship", "because", ]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('cleaned_words') \
     .setStopWords(stop_words)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Finisher --> converts the cleaned words into a format easier to use

#https://sparknlp.org/api/python/reference/autosummary/sparknlp/base/finisher/index.html

finisher = Finisher() \
     .setInputCols(['cleaned_words']) \
     .setOutputCols("finished_tokens")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#Now, put everything together to create a pipeline that processes the text.

pipeline = Pipeline() \
     .setStages([documentAssembler_text,                  
                 tokenizer,
                 normalizer,                  
                 lemmatizer,                  
                 stopwords_cleaner, 
                 finisher])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Now, apply pipeline to the data to transform it in the needed way for the LDA

processed_text = pipeline.fit(df).transform(df)
processed_text.persist()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[id: string, created: string, author: string, score: string, title: string, selftext: string, num_comments: string, entire_text: string, year: int, person_1: string, person_2: string, finished_tokens: array<string>]

In [17]:
#Ensure that dataframe is persisted in memory
processed_text.storageLevel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StorageLevel(True, True, False, False, 1)

## Vectorization 

In this next step, I'll count the frequency for each word and perform a TF-IDF measure, to be able to identify which words are really salient

In [18]:
tfizer = CountVectorizer(inputCol='finished_tokens', outputCol='terms_frequencies', minDF=0.01, maxDF=0.80)
tf_model = tfizer.fit(processed_text)
tf_result = tf_model.transform(processed_text)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
processed_text.unpersist() #Unpersist to release memory and persist another df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[id: string, created: string, author: string, score: string, title: string, selftext: string, num_comments: string, entire_text: string, year: int, person_1: string, person_2: string, finished_tokens: array<string>]

In [20]:
idfizer = IDF(inputCol='terms_frequencies', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## LDA 

In [21]:
tfidf_result.persist() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[id: string, created: string, author: string, score: string, title: string, selftext: string, num_comments: string, entire_text: string, year: int, person_1: string, person_2: string, finished_tokens: array<string>, terms_frequencies: vector, tf_idf_features: vector]

In [22]:
num_topics = 15 
max_iter = 10

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features', seed=25)
lda_model = lda.fit(tfidf_result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
tfidf_result.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[id: string, created: string, author: string, score: string, title: string, selftext: string, num_comments: string, entire_text: string, year: int, person_1: string, person_2: string, finished_tokens: array<string>, terms_frequencies: vector, tf_idf_features: vector]

In [24]:
#This code is copied from the tutorial https://colab.research.google.com/github/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb#scrollTo=XwOdzQ_PAJi0

vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#This code is copied from the tutorial https://colab.research.google.com/github/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb#scrollTo=XwOdzQ_PAJi0
num_top_words = 10

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------------------------------------------------------------------------+
|topic|                                                                 topicWords|
+-----+---------------------------------------------------------------------------+
|    0|    [love, ex, break, feel, cheat, hurt, boyfriend, bad, issue, girlfriend]|
|    1|            [porn, wedding, u, ring, say, fight, drink, tell, thing, watch]|
|    2|            [block, ex, friend, fuck, tell, gf, back, message, then, break]|
|    3|           [ampxb, people, friend, woman, man, guy, very, make, girl, feel]|
|    4|                 [say, tell, then, day, night, ask, go, drink, party, home]|
|    5|               [dog, fiancé, sleep, cat, call, wake, bed, room, lie, phone]|
|    6|      [husband, partner, kid, marry, son, marriage, work, child, job, live]|
|    7|       [brother, game, play, birthday, drug, parent, gift, video, pay, day]|
|    8|       [friend, girl, feelings, date, crush, love, guy, really, feel,

## Add column with topic distribution for each submission

In [26]:
# Transform data to get topic distributions
df_with_lda = lda_model.transform(tfidf_result)

df_with_lda.select("topicDistribution").show(1, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topicDistribution                                                                                                                                                                                                                                                                                                         |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[2.390090226125229E-4,2.3245682967455535E-4,2.30

In [27]:
df_repartitioned = df_with_lda.repartition(10)
df_repartitioned.write.parquet("s3://finalproject-nat-s3/data_withtopics", mode = 'overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Interpretation of topics

Some topics seem reasonable and easier to interpret, whereas other ones are vague. I wish I could have fine tuned more the model using performance metrics, but the limitation of resources made the modeling very slow and it reached a point where I was not able to invest more time fine-tuning it. 

Therefore, I've decided to advance the analysis with the available data, stick to the reasonable topics and categorize the rest as "Unclear". 

The topics I'll use for further analysis are labeled as following:

- Topic 0: Romantic relationships 
- Topic 8: Dating
- Topic 9: Finances, employment and housing
- Topic 10: Social media and messaging
- Topic 11: Family

P.D. Although topic 6 seems to be about marriage and kids, after exploring different text samples I came to the conclusion that it is not a coherent category and therefore, I'll label it as unclear as well. 
